<a href="https://colab.research.google.com/github/Tako-San/ml-course/blob/main/t1/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>